3.9 多层感知机的从零开始实现
我们已经从上一节里了解了多层感知机的原理。下面，我们一起来动手实现一个多层感知机。首先导入实现所需的包或模块。

In [14]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_tensorflow
import d2lzh_tensorflow2 as d2l
print(tf.__version__)

2.18.0


3.9.1 获取和读取数据
这里继续使用Fashion-MNIST数据集。我们将使用多层感知机对图像进行分类

In [15]:
# 加载数据集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 设置批量大小
batch_size = 256

# 将数据类型转换为float32
x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

# 将像素值归一化到0-1之间
x_train = x_train / 255.0
x_test = x_test / 255.0

# 创建训练数据迭代器
train_iter = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
# 创建测试数据迭代器
test_iter = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

3.9.2 定义模型参数
我们在3.6节（softmax回归的从零开始实现）里已经介绍了，Fashion-MNIST数据集中图像形状为 28×28，类别数为10。本节中我们依然使用长度为 28×28=784 的向量表示每一张图像。因此，输入个数为784，输出个数为10。实验中，我们设超参数隐藏单元个数为256。

In [16]:
# 定义模型参数
num_inputs, num_outputs, num_hiddens = 784, 10, 256

# 初始化第一层权重和偏置
W1 = tf.Variable(tf.random.normal(shape=(num_inputs, num_hiddens),
                                  mean=0, stddev=0.01, dtype=tf.float32))
b1 = tf.Variable(tf.zeros(num_hiddens, dtype=tf.float32))

# 初始化第二层权重和偏置
W2 = tf.Variable(tf.random.normal(shape=(num_hiddens, num_outputs),
                                  mean=0, stddev=0.01, dtype=tf.float32))
b2 = tf.Variable(tf.random.normal([num_outputs], stddev=0.1))

3.9.3 定义激活函数
这里我们使用基础的max函数来实现ReLU，而非直接调用relu函数。

In [17]:
def relu(x):
    return tf.math.maximum(x,0)

In [18]:
def net(X):
    # 将输入X重塑为二维张量
    # 第一维设为-1,表示批量大小, 允许自动计算以适应不同大小的批次
    # 第二维设为num_inputs, 明确指定输入特征的数量
    X = tf.reshape(X, shape=[-1, num_inputs])
    
    # 计算第一层的输出，应用ReLU激活函数
    h = relu(tf.matmul(X, W1) + b1)
    
    # 计算第二层的输出，并应用softmax函数得到概率分布
    return tf.math.softmax(tf.matmul(h, W2) + b2)

3.9.5. 定义损失函数¶
为了得到更好的数值稳定性，我们直接使用Tensorflow提供的包括softmax运算和交叉熵损失计算的函数。

In [19]:
def loss(y_hat,y_true):
    return tf.losses.sparse_categorical_crossentropy(y_true,y_hat)

3.9.6. 训练模型

In [20]:
num_epochs, lr = 5, 0.5

In [21]:
num_epochs, lr = 5, 0.5
params = [W1, b1, W2, b2]
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)

2024-10-27 18:04:59.550770: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-10-27 18:04:59.633356: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epoch 1, loss 0.7856, train acc 0.705, test acc 0.819


2024-10-27 18:05:00.851144: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epoch 2, loss 0.4862, train acc 0.820, test acc 0.837
epoch 3, loss 0.4220, train acc 0.842, test acc 0.849


2024-10-27 18:05:03.167800: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epoch 4, loss 0.3883, train acc 0.855, test acc 0.857
epoch 5, loss 0.3663, train acc 0.865, test acc 0.861
